# Interactive use of PAHFIT and cube fitting

*it would be nice if we could run pahfit-cube in a notebook, and instantly see the results. But since parallel processing can be so finicky, I'll just focus on fitting one selected region.*

To run PAHFIT on the entire cube, we can copy and adapt some code from https://github.com/drvdputt/PAHFIT-cube. A good goal for this notebook would be to take inspiration from the cube_fitting notebook, and come up with some interesting visualizations for the full-cube PAHFIT results.

In [ ]:
%matplotlib inline
from specutils import Spectrum1D
from astropy.nddata import StdDevUncertainty
from astropy.io import fits
from astropy import wcs
from jdaviz import Cubeviz
from photutils.aperture import (CircularAperture, aperture_photometry)
from matplotlib import pyplot as plt
import numpy as np
import pahfit.helpers
from astropy import units as u

def plot_spectrum1d(spec1d, ax=None, **kwargs):
    if ax is None:
        f, (ax1) = plt.subplots(1, 1, figsize=(15, 5))
    else:
        ax1 = ax
    ax1.set_xlabel("Observed Wavelength (microns)")  
    ax1.set_ylabel(f"flux ({spec1d.flux.unit})")
    ax1.plot(spec1d.spectral_axis.to(u.micron).value, spec1d.flux.value, **kwargs)
    ax1.legend()
    return ax1

In [ ]:
# MRS science data cubes
# multiple given here, to make it easier to compare for testing stuff
filenames = ["stage3-boris/BandCube_ch1-long_s3d.fits", "stage3-dries/BandCube_ch1-long_s3d.fits", "spec3-steps-dries/lowres_ch1-long_s3d.fits"]

In [ ]:
cubeviz = Cubeviz()
cubeviz.app

In [ ]:
[cubeviz.app.load_data(f) for f in filenames]
#cubeviz.app.load_data(filename2)

## 1D spectrum
I made a minimal PAHFIT science pack, that goes with one of the MIRI cubes we used for the hack week. 

By importing PAHFIT, we can use one of the utility functions to load the science pack and construct the corresponding model. The goal of this part is to show that PAHFIT can be used interactively, allowing us to quickly judge how good the chosen science pack is working. This is useful for PAHFIT development, e.g. exploring for which regions of the PDR the science pack is working.

In [ ]:
# CircularAperture uses xy pixels
cubeviz.app.get_viewer_reference_names()

In [ ]:
# regions = cubeviz.app.get_subsets_from_viewer('flux-viewer')
# region1_exists = 'region 1' in regions
# if region1_exists:
#     x0, y0 = region1.center.x, region1.center.y
#     center_xy = [x0, y0]   
#     r_pix = region1.radius
# else:
#     print("Setting default circular aperture in the center of the map with a radius of 10 pixels")
#     # RC: we could define our own region(s) programmatically based on this instead of clicking
#     nx, ny = cubeviz.app.data_collection[0].shape[1:]
#     x0, y0 = nx/2, ny/2
#     center_xy = [x0, y0]   
#     r_pix = 6
    
# aperture = CircularAperture(center_xy, r=r_pix)
# print(aperture)

# spec1d_len = len(spec1d.wavelength)
# circle_sum = []
# circle_sum_unc = []
# for idx in range(spec1d_len):
#     slice2d = spec1d.data[:, :, idx]
#     phot_table = aperture_photometry(slice2d, aperture, wcs=w.celestial, method='exact')
#     circle_sum.append(phot_table['aperture_sum'][0])
    
#     slice2d_unc = spec1d.uncertainty.array[:, :, idx]
#     phot_table = aperture_photometry(slice2d_unc, aperture, wcs=w.celestial, method='exact')
#     circle_sum_unc.append(np.sqrt(phot_table['aperture_sum'][0]))

# # put result in Spectrum1D format
# wav = spec1d.wavelength.to(u.micron)
# unc = StdDevUncertainty(np.array(circle_sum_unc))

# circle_spec = Spectrum1D(flux = np.array(circle_sum) * spec1d.flux.unit, spectral_axis=wav, uncertainty=unc)
# plot_spectrum1d(circle_spec, label='Circle at (x,y,r)=(%.2f,%.2f,%.2f)'% (x0, y0, r_pix))
# circle_spec.wavelength

# the above code extracts a region. Don't need it for now.

## Use new trimmed model from PAHFIT


In [ ]:
# Define observational data. Fake the uncertainty for now 
# (getting it from the JWST cube would be a good thing to ask about)
print(filenames)
specs = [Spectrum1D.read(f) for f in filenames]
specs[0].shape

In [ ]:
from make_trimmed_model import make_trimmed_model
def fit(spec):
    # take central pixel
    ny, nx, nw = spec.shape
    spec1d = spec[ny // 2, nx // 4]
    obsdata = {"x": spec1d.spectral_axis.to(u.micron), "y": spec1d.flux, "unc": spec1d.uncertainty.quantity}
    print(obsdata)
    model_base = make_trimmed_model('scipack_ExGal_SpitzerIRSSLLL.ipac', obsdata)
    model_result = pahfit.helpers.fit_spectrum(obsdata, model_base, maxiter=1000)
    print(model_result)
    # plot separate components
    fig, ax = plt.subplots(2, 1, figsize=(10,10))
    model_base.plot(ax, obsdata['x'], obsdata['y'], obsdata['unc'], model_result)

In [ ]:
fit(specs[2])